In [89]:
import csv
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import time
import plotly.express as px
from selenium import webdriver


In [90]:
driver = webdriver.Chrome()

### Fonction pour générer l'URL en fonction du terme recherché

In [91]:
def url_by_term(search_term,num_page):
    form = "https://www.amazon.com/s?k={}&ref=nb_sb_noss_1&page="+str(num_page)
    term = search_term.replace(" ", "+")
    return form.format(term)

term = "ordinateur portable"


### Extraction des liens

In [92]:
#Ici j'extraie les liens de redirections
links = []
for i in range(20):
    
    url = url_by_term(term,i)
    driver.get(url)

    bs = BeautifulSoup(driver.page_source,'html.parser')
    results = bs.find_all('h2', {'class':'a-size-mini a-spacing-none a-color-base s-line-clamp-2'})
    

    for r in results:
        links.append('https://www.amazon.com'+r.find('a')['href'])

#Ici j'extraie les vrais lien à partir des liens de redirections
newlink = []
for x in links: 
    
    driver.get(x)
    newlink.append(driver.current_url)
    
   
    

    

### Extraction d'avis et de commentaire

In [93]:

for j in range(len(newlink)):
    newlink[j] = newlink[j].replace("dp","product-reviews")
    
review = []    
list_links=[]
positif=[]
for g in newlink:
    for i in range(5):
        for j in ['positive','critical']:
            list_links.append(g+"&filterByStar="+j+"&pageNumber="+str(i))
            #driver.get(g+"&filterByStar="+j+"&pageNumber="+str(i))
            driver.get(g+"&filterByStar="+j+"&pageNumber="+str(i))

            bs = BeautifulSoup(driver.page_source,'html.parser')
            results = bs.find_all('span', {'data-hook':'review-body'})


            for r in results:
                review.append(r.find('span').text)


                if j == 'positive':
                    positif.append(1)
                if j == 'critical':
                    positif.append(0)



### Raccourcis des commentaires (Uniquement première et dernière phrase)

In [94]:
review_list = []
for i in review:
    
    liste= re.split('[?.!;]', i)
    
    if len(liste)>2:
        j = liste[0]+","+liste[-2]
    else:
        j = liste[0]+","+liste[-1]
    review_list.append(j)
    
review_list

['\n  6300 hundred processor is few generation old,  It should be priced 350 or 399',
 '\n  Product DOA, What is the use of of a computer that does not boot up',
 '\n  did not last 1 year, hardware failure,\n',
 '\n  Item faulty, Doesn’t work',
 '\n  6300 hundred processor is few generation old,  It should be priced 350 or 399',
 '\n  Product DOA, What is the use of of a computer that does not boot up',
 '\n  did not last 1 year, hardware failure,\n',
 '\n  Item faulty, Doesn’t work',
 '\n  I ordered this yesterday and got it within 24 hours, I love it',
 '\n  I bought this to replace a 2012 Mac Mini in our kitchen,It was over $500 - I wish it was a little cheaper, maybe $400, but at least it should last me 5+ years if nothing dies and I can easily use it with a TV if I want to change my use case',
 '\n  So easy to use compared to Window  PC, I wanted to replace my All in one PC and the Chromebox with a monitor and speakers did the Job',
 "\n  I bought this to play GeForce Now on since

### Nettoyage et Mise en Format JSON

In [95]:

d = {'review' : review_list, 'positive': positif}
review_df = pd.DataFrame(data=d)
review_df = review_df.replace('\n','', regex=True)
review_df = review_df.replace('\u2019','`', regex=True)
review_df = review_df.replace('\u201d','', regex=True)
review_df = review_df.replace('\u201c','', regex=True)
review_df = review_df.replace('\/',' ', regex=True)

review_df.drop_duplicates(keep = 'first', inplace=True)

review_df.to_json(r'C:\Users\cleme\OneDrive\Documents\Python Scripts\Amazon Web Scrapping\review_amazon_'+term.replace(' ','')+str(time.strftime("%d_%B_%Y"))+'.json',orient='records')